In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import os
#os.chdir("/home/kaborg15/Vibrent_Dataset_Collection/")
os.chdir(r"C:\Users\kaborg15\Python_projects\Vibrent_Dataset_Collection")



In [3]:
import pandas as pd
from src import retrieve_image_bucket_data
from resources.constants import *

picture_triplets_df = pd.read_csv(DATASET_FOLDER + "picture_triplets.csv", sep=";")

dataset_picture_ids = picture_triplets_df["picture.id"].unique()
#print(len(dataset_picture_ids), len(pictures_df["id"].unique()))
IMAGES_FOLDER = "resources/data/dataset/images/"
existing_images = os.listdir(IMAGES_FOLDER)
dataset_picture_ids = list(set(dataset_picture_ids) - set(existing_images))
retrieve_image_bucket_data.download_to_folder(dataset_picture_ids, IMAGES_FOLDER)

KeyboardInterrupt: 

In [11]:
from src import create_image_embeddings
import pandas as pd
from resources.constants import *

picture_triplets_df = pd.read_csv(DATASET_FOLDER + "picture_triplets.csv", sep=";")
picture_triplets_df[:10]

,picture.id,outfit.id,displayOrder
0,picture.0000cdba64314d84a49ed1c266589cc0,outfit.794483397da8425a813301eecf9828c6,0
1,picture.00058abb53434872ae9bb4270ae21f8e,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,2
2,picture.00063f52c36d43ada95da45f819b30b4,outfit.9fd1c42c3db543c5b6e53b0db1ee8c0f,3
3,picture.0008443461814f5c988f123718bbd20e,outfit.a7539783b6e94591bdf4e10339afc1d7,3
4,picture.000a5db3362049aebcc1eb2bf7bde95f,outfit.745fa2bc8156478bac6c0f7d46dadbda,1
5,picture.000cf715019a4a02aaf0dc479212195a,outfit.f11bdc25d87946f8b831b608bc8fb574,1
6,picture.000ddec26b7a4aa495e2d7db9e9585e9,outfit.40d4e8f739a74e488769b16f8f87ca83,3
7,picture.0010c2e161154d6893734981d5455e76,outfit.9387d05b47f906c5,0
8,picture.0013342f663843d4b7397b4b88b98e6d,outfit.b24fc57f02ac4e4c9061761ca18d9d3c,0
9,picture.00135ce8b1a04c5daa60cf7bdd99bcd5,outfit.12ae28ab0dc5494c98a1de2f8ce04b79,1


In [14]:
embeddings_df = create_image_embeddings.get_df_image_embeddings(picture_triplets_df[:10], DATASET_IMAGE_EMBEDDINGS_FOLDER, config_class=create_image_embeddings.EfficientNet_V2_L_final)

PKL name: EfficientNet_V2_L_final.pkl | Model: EfficientNet | Weights: EfficientNet_V2_L_Weights | IMAGENET1K_V1
Generating embeddings from EfficientNet_V2_L_final to EfficientNet_V2_L_final.pkl...


  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
embeddings_df["embeddings"].values[0].numpy()

array([-0.109  , -0.08044, -0.1355 , ...,  0.2273 , -0.063  , -0.10565],
      dtype=float16)